## 0. Импорты и установки

In [1]:
!pip install corus

In [2]:
!pip install pymystem3 pymorphy2

In [3]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-vn55uhia
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-vn55uhia
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done


In [4]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  51.1M      0  0:00:27  0:00:27 --:--:-- 59.4M


In [5]:
!wget https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu

--2023-12-07 20:14:22--  https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu [following]
--2023-12-07 20:14:22--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14704579 (14M) [text/plain]
Saving to: ‘ru_syntagrus-ud-dev.conllu.1’

ru_syntagrus-ud-dev 100%[===================>]  14.02M  --.-KB/s    in 0.1s    

2023

In [6]:
# библиотеки для работы с эмбеддингами
import adagram
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100')

# для корпуса
from corus import load_ud_syntag
path = 'ru_syntagrus-ud-dev.conllu'

# обработка данных и ML
import json
import pandas as pd
from lxml import html
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import cosine
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
nltk.download('stopwords')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Выбор неоднозначных лексем
Выбираю 5 лексем: рука, база, брус, ключ, повод

Словарь [MAC](https://lexicography.online/explanatory/mas/), который нормально парсится, для парсинга использовала selectolax, но он не запустился в коллабе, поэтому код сделала отдельным .ру файлом, а значения сохранила в json

In [7]:
lex = ['рука', 'база', 'голова', 'круг', 'повод']

In [8]:
with open('meanings.json', 'r', encoding='utf-8') as f:
        meanings = json.load(f)

In [9]:
def pretty_print_dictionary(dictionary):
    for key, value in dictionary.items():
        if isinstance(value, dict):
            print(f"{key}:")
            pretty_print_dictionary(value)
        else:
            print(f"{key}: {value}")

pretty_print_dictionary(meanings)

рука: ['Каждая из двух верхних конечностей человека от плечевого сустава до кончиков пальцев. ', 'Употребляется как символ орудия труда человека, а также как символ самого труда, деятельности человека. ', 'Манера письма, почерк. ', 'Употребляется в значении: работники, люди, выполняющие какую-л. работу. ', 'Употребляется для обозначения человека как обладателя, владельца чего-л. ', 'Употребляется для обозначения человека, который имеет власть, влияние, может оказать покровительство, содействие кому-л. ', 'Употребляется в некоторых сочетаниях в значении: согласие на замужество, готовность вступить в брак. ', 'Место, очередь отдельного участника в чем-л. (в общей работе, в какой-л. игре и т. п.), а также отдельный участник, вступающий последовательно за кем-л. ', 'Употребляется в значении: того или иного вида, сорта, качества и т. п. ', 'В вин. п. ед. ч. с предлогом „под“ и определением, указывающим на состояние, настроение, означает: в таком состоянии, настроении. ']
база: ['Нижняя, бол

## 2. Сбор корпуса
Беру корпус [SynTagRus](https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-dev.conllu), который уже размечен в UD

In [10]:
def get_texts(path, words):
    res = {word: [] for word in words}
    records = load_ud_syntag(path)
    for record in records:
        for token in record.tokens:
            if token.lemma in words:
                if record not in res[token.lemma]:
                    res[token.lemma].append(record)

    return res

In [11]:
corpus = get_texts(path, lex)

In [12]:
for word in lex:
  print(f'Колиечство предложений со словом {word} в корпусе: {len(corpus[word])}')

Колиечство предложений со словом рука в корпусе: 71
Колиечство предложений со словом база в корпусе: 22
Колиечство предложений со словом голова в корпусе: 42
Колиечство предложений со словом круг в корпусе: 15
Колиечство предложений со словом повод в корпусе: 27


## 3. Дезамбигуация контекстов с помощью Adagram
(с ELMo ничего не получилось)

In [13]:
for word in lex:
    print(word, vm.word_sense_probs(word))

рука [(1, 0.269292101473561), (2, 0.21868880491977258), (3, 0.174332601644788), (4, 0.1874482738053162), (5, 0.14978834588544554)]
база [(0, 0.2749814291520308), (1, 0.22281705240672087), (2, 0.22811624166900427), (4, 0.2740804992298799)]
голова [(0, 0.15136618792493253), (2, 0.18463972125993772), (3, 0.23746755179898937), (4, 0.031868947553770864), (5, 0.18276442527655343), (6, 0.21124967139097275)]
круг [(0, 0.362903810933423), (1, 0.20540961127084062), (2, 0.059090420714983015), (3, 0.37259581261023217)]
повод [(0, 0.30422305631537927), (1, 0.2768945605956191), (2, 0.18743370890242153), (3, 0.1881764008922709), (4, 0.043271967409987344)]


In [14]:
def lemmatized_context(s):
    return [token.lemma for token in s.tokens if token.pos != 'PUNCT']

def disambiguate_context(model, word, context):
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()

In [15]:
adam_corpus = {}
for word in lex:
    adam_corpus[word] = []
    for text in corpus[word]:
        meaning = disambiguate_context(vm, word, text)
        adam_corpus[word].append([text, meaning])

In [16]:
pretty_print_dictionary(adam_corpus)

рука: [[UDSent(id='2011Alpinizm.xml_200', text='Лазание наиболее безопасно в тех случаях, когда лезущий соблюдает правило трех точек опоры: для двух рук и ноги или обеих ног и руки.', attrs={}, tokens=[UDToken(id='1', text='Лазание', lemma='лазанье', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Neut', 'Number': 'Sing'}, head_id='3', rel='nsubj'), UDToken(id='2', text='наиболее', lemma='наиболее', pos='ADV', feats={'Degree': 'Pos'}, head_id='3', rel='obl'), UDToken(id='3', text='безопасно', lemma='безопасный', pos='ADJ', feats={'Degree': 'Pos', 'Gender': 'Neut', 'Number': 'Sing', 'Variant': 'Short'}, head_id='0', rel='root'), UDToken(id='4', text='в', lemma='в', pos='ADP', feats={}, head_id='6', rel='case'), UDToken(id='5', text='тех', lemma='тот', pos='DET', feats={'Case': 'Loc', 'Number': 'Plur', 'PronType': 'Dem'}, head_id='6', rel='det'), UDToken(id='6', text='случаях', lemma='случай', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Masc', 'Number'

Также делаю со значениями, только приходится делать свою токенизацию/лемматизацию

In [17]:
mystem = Mystem()

def lemmatized_meaning(s):
     return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

def disambiguate_meaning(model, word, context):
    word, _ = lemmatized_meaning(word)
    probs = model.disambiguate(word, lemmatized_meaning(context))
    return 1 + probs.argmax()

In [18]:
adam_corpus_for_meanings = {}
for word in lex:
    adam_corpus_for_meanings[word] = []
    for text in meanings[word]:
        meaning = disambiguate_meaning(vm, word, text)
        adam_corpus_for_meanings[word].append([text, meaning])

In [19]:
pretty_print_dictionary(adam_corpus_for_meanings)

рука: [['Каждая из двух верхних конечностей человека от плечевого сустава до кончиков пальцев. ', 2], ['Употребляется как символ орудия труда человека, а также как символ самого труда, деятельности человека. ', 6], ['Манера письма, почерк. ', 6], ['Употребляется в значении: работники, люди, выполняющие какую-л. работу. ', 6], ['Употребляется для обозначения человека как обладателя, владельца чего-л. ', 6], ['Употребляется для обозначения человека, который имеет власть, влияние, может оказать покровительство, содействие кому-л. ', 6], ['Употребляется в некоторых сочетаниях в значении: согласие на замужество, готовность вступить в брак. ', 6], ['Место, очередь отдельного участника в чем-л. (в общей работе, в какой-л. игре и т. п.), а также отдельный участник, вступающий последовательно за кем-л. ', 6], ['Употребляется в значении: того или иного вида, сорта, качества и т. п. ', 6], ['В вин. п. ед. ч. с предлогом „под“ и определением, указывающим на состояние, настроение, означает: в таком

Для руки Adagram разметил по контекстам два значения 2 и 6, для базы - разметил одно, для головы - 3, для круга - 2 и для повода - 2. Рассмотрим тогда контексты без базы, потому что Adagram не различает их.

Перепишем словарь и заполним его пустыми значениями тоже, который адаграм выделил сам

In [20]:
lex_new = ['рука', 'голова', 'круг', 'повод']

In [21]:
dictionary = {}
for word in lex_new:
    dictionary[word] = {}
    for x in range(8):
        dictionary[word][x] = []
    for meanings in adam_corpus_for_meanings[word]:
        dictionary[word][meanings[1]].append(meanings[0])

pretty_print_dictionary(dictionary)

рука:
0: []
1: []
2: ['Каждая из двух верхних конечностей человека от плечевого сустава до кончиков пальцев. ']
3: []
4: []
5: []
6: ['Употребляется как символ орудия труда человека, а также как символ самого труда, деятельности человека. ', 'Манера письма, почерк. ', 'Употребляется в значении: работники, люди, выполняющие какую-л. работу. ', 'Употребляется для обозначения человека как обладателя, владельца чего-л. ', 'Употребляется для обозначения человека, который имеет власть, влияние, может оказать покровительство, содействие кому-л. ', 'Употребляется в некоторых сочетаниях в значении: согласие на замужество, готовность вступить в брак. ', 'Место, очередь отдельного участника в чем-л. (в общей работе, в какой-л. игре и т. п.), а также отдельный участник, вступающий последовательно за кем-л. ', 'Употребляется в значении: того или иного вида, сорта, качества и т. п. ', 'В вин. п. ед. ч. с предлогом „под“ и определением, указывающим на состояние, настроение, означает: в таком состояни

In [22]:
answer = []
for word in lex_new:
    print('\n', word, '\n')
    for text in adam_corpus[word][0:5]:
        print('Текст: ', text[0].text, '\nЗначение: ', *dictionary[word][text[1]])
        answer.append(int(input('Совпадает? ')))


 рука 

Текст:  Лазание наиболее безопасно в тех случаях, когда лезущий соблюдает правило трех точек опоры: для двух рук и ноги или обеих ног и руки. 
Значение:  Каждая из двух верхних конечностей человека от плечевого сустава до кончиков пальцев. 
Совпадает? 1
Текст:  В зависимости от общего объема капитала, имеющегося в распоряжении отдельного социального "игрока", могут возрастать или падать его "ставки", более или менее успешно складываться его движение в социальном пространстве; в зависимости от структуры капитала, находящегося в руках данного игрока, осуществляется выбор его социальной траектории (т. е., по сути, выбор жизненного пути). 
Значение:  Употребляется как символ орудия труда человека, а также как символ самого труда, деятельности человека.  Манера письма, почерк.  Употребляется в значении: работники, люди, выполняющие какую-л. работу.  Употребляется для обозначения человека как обладателя, владельца чего-л.  Употребляется для обозначения человека, который имеет власть

In [24]:
print('Accuracy: ', sum(answer)/len(answer))

Accuracy:  0.4


Поскольку сравнить мне не с чем (ELMo не заработал), просто опишу, что получилось с Адаграм. У него была бы хорошая метрика, если бы лучше кластеризовались значения (потому что было достаточно много контекстов, где нет подходящего значения в словаре). А ошибок там, где значения всё-таки есть в словаре, не так много.